In [107]:
!pip install pymorphy2
!pip install transformers
!pip install fuzzywuzzy
!pip install pytelegrambotapi
!pip install langchain pandas tiktoken huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00


In [81]:
# для мешка слов и TF-IDF импортируем:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
nltk.download('punkt') # для токенизации необходимо скачать модуль
import pymorphy2 # библиотека для лемматизации

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [82]:
from google.colab import drive
import google.colab as colab
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [126]:
codex = "/content/gdrive/MyDrive/30092023хакатон/code.txt"
import pandas as pd
#data_read = open(codex,"r")
#data = data_read.read()
df = pd.read_csv("/content/gdrive/MyDrive/30092023хакатон/data.csv")
print(df)
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# грузим фрейм в лоадер, выделив колонку для векторизации (здесь может быть место для дискуссий)
loader = DataFrameLoader(df, page_content_column='question')
documents = loader.load()
print(documents[0])



               Datetime                                question  \
0   2023-09-29 02:33:51                             who are you   
1   2023-09-29 02:35:36                                  кто ты   
2   2023-09-29 02:35:43                                   кто я   
3   2023-09-29 02:43:09                                     кто   
4   2023-09-29 02:43:27                                     что   
5   2023-09-29 02:43:53                                  тко ты   
6   2023-09-29 02:48:38                          что происходит   
7   2023-09-29 14:40:00                                     а ?   
8   2023-09-29 14:45:13  сколько мне полагается отпускных дней?   
9   2023-09-29 15:02:51                 Что такое Смартократия?   
10  2023-09-29 17:02:55                                       k   
11  2023-09-29 17:22:30                                  отпуск   
12  2023-09-29 19:02:04                          какой вопрос ?   
13  2023-09-29 19:03:30                            что делать 

In [84]:
import nltk
nltk.download('punkt')
sen = nltk.sent_tokenize(data)
print(sen)

['# Преамбула\n\nСмартократия - Интегральная Политика Группы Компаний Smart Consulting (далее - ГК).', 'Базируется на следующих ключевых принципах:\n\n1.', '**Принцип результативности**\nВкладываем время, энергию и ресурсы только в то, что приближает к целям ГК.', '2.', '**Принцип консента**\nЛюбая инициатива имеет право на осуществление, пока не доказан ущерб от нее.', '3.', '**Принцип практики**\nВсе гипотезы мы проверяем на практике.', '4.', '**Принцип непрерывных улучшений**\nУлучшаем себя на основе регулярной обратной связи и рефлексии.', '5.', '**Принцип равноценности**\nКаждый на равных может участвовать в подготовке решений, которые его затрагивают.', '6.', '**Принцип «все видят все»**\nВся информация в ГК общедоступна, если нет причин для конфиденциальности.', '7.', '**Принцип ответственности**\nКаждый сотрудник отвечает за результат ГК и может на него влиять.', 'Кодекс Смартократии (далее – Кодекс) определяет основные понятия, правила и процессы управления, обладающие приорит

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [85]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [86]:
import pymorphy2
import re
from nltk.corpus import stopwords
morph = pymorphy2.MorphAnalyzer()
tokenized = []
for i in sen:
  tok_sen = ''
  txt = re.findall(r'[а-я]+', i.lower())
  for j in txt:
    if j not in stopwords.words('russian'):
      w = morph.parse(j)[0].normal_form
      if tok_sen == '':
        tok_sen += w
      else:
        tok_sen += (' ' + w)
  tokenized.append(tok_sen)

print(tokenized)

['преамбула смартократия интегральный политика группа компания далее гк', 'базироваться следующий ключевой принцип', 'принцип результативность вкладывать время энергия ресурс приближать цель гк', '', 'принцип консента любой инициатива иметь право осуществление пока доказать ущерб', '', 'принцип практика гипотеза проверять практика', '', 'принцип непрерывный улучшение улучшать основа регулярный обратный связь рефлексия', '', 'принцип равноценность каждый равный участвовать подготовка решение который затрагивать', '', 'принцип видеть весь информация гк общедоступный причина конфиденциальность', '', 'принцип ответственность каждый сотрудник отвечать результат гк влиять', 'кодекс смартократия далее кодекс определять основной понятие правило процесс управление обладать приоритетный право применение иной правило управление принимать гк', 'сотрудник гк далее сотрудник мочь полагаться полномочие дать это кодекс также соглашаться регламентировать деятельность соответствие прописать н м обязанно

In [87]:
sen

['# Преамбула\n\nСмартократия - Интегральная Политика Группы Компаний Smart Consulting (далее - ГК).',
 'Базируется на следующих ключевых принципах:\n\n1.',
 '**Принцип результативности**\nВкладываем время, энергию и ресурсы только в то, что приближает к целям ГК.',
 '2.',
 '**Принцип консента**\nЛюбая инициатива имеет право на осуществление, пока не доказан ущерб от нее.',
 '3.',
 '**Принцип практики**\nВсе гипотезы мы проверяем на практике.',
 '4.',
 '**Принцип непрерывных улучшений**\nУлучшаем себя на основе регулярной обратной связи и рефлексии.',
 '5.',
 '**Принцип равноценности**\nКаждый на равных может участвовать в подготовке решений, которые его затрагивают.',
 '6.',
 '**Принцип «все видят все»**\nВся информация в ГК общедоступна, если нет причин для конфиденциальности.',
 '7.',
 '**Принцип ответственности**\nКаждый сотрудник отвечает за результат ГК и может на него влиять.',
 'Кодекс Смартократии (далее – Кодекс) определяет основные понятия, правила и процессы управления, обл

In [88]:
sen[0].lower()

'# преамбула\n\nсмартократия - интегральная политика группы компаний smart consulting (далее - гк).'

In [89]:
import re
txt = re.findall(r'[а-я]+', sen[0].lower())
txt

['преамбула',
 'смартократия',
 'интегральная',
 'политика',
 'группы',
 'компаний',
 'далее',
 'гк']

In [90]:
stopwords.words('russian')

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()



In [92]:
bag = vectorizer.fit_transform(tokenized)

In [93]:
print(vectorizer.vocabulary_)

print(bag.toarray())

{'преамбула': 458, 'смартократия': 588, 'интегральный': 188, 'политика': 429, 'группа': 99, 'компания': 221, 'далее': 102, 'гк': 89, 'базироваться': 15, 'следующий': 583, 'ключевой': 213, 'принцип': 481, 'результативность': 543, 'вкладывать': 42, 'время': 70, 'энергия': 715, 'ресурс': 545, 'приближать': 471, 'цель': 702, 'консента': 227, 'любой': 257, 'инициатива': 183, 'иметь': 180, 'право': 455, 'осуществление': 387, 'пока': 422, 'доказать': 120, 'ущерб': 680, 'практика': 457, 'гипотеза': 88, 'проверять': 491, 'непрерывный': 318, 'улучшение': 659, 'улучшать': 658, 'основа': 380, 'регулярный': 540, 'обратный': 333, 'связь': 567, 'рефлексия': 547, 'равноценность': 518, 'каждый': 203, 'равный': 519, 'участвовать': 675, 'подготовка': 417, 'решение': 550, 'который': 236, 'затрагивать': 162, 'видеть': 39, 'весь': 32, 'информация': 192, 'общедоступный': 335, 'причина': 488, 'конфиденциальность': 233, 'ответственность': 390, 'сотрудник': 612, 'отвечать': 391, 'результат': 542, 'влиять': 47, 

In [94]:
print(bag)

  (0, 458)	1
  (0, 588)	1
  (0, 188)	1
  (0, 429)	1
  (0, 99)	1
  (0, 221)	1
  (0, 102)	1
  (0, 89)	1
  (1, 15)	1
  (1, 583)	1
  (1, 213)	1
  (1, 481)	1
  (2, 89)	1
  (2, 481)	1
  (2, 543)	1
  (2, 42)	1
  (2, 70)	1
  (2, 715)	1
  (2, 545)	1
  (2, 471)	1
  (2, 702)	1
  (4, 481)	1
  (4, 227)	1
  (4, 257)	1
  (4, 183)	1
  :	:
  (185, 308)	1
  (185, 69)	2
  (185, 450)	1
  (185, 564)	1
  (185, 629)	1
  (185, 389)	1
  (185, 208)	1
  (185, 215)	1
  (185, 179)	1
  (185, 423)	1
  (185, 477)	1
  (185, 54)	1
  (185, 709)	1
  (185, 592)	1
  (185, 573)	1
  (185, 321)	1
  (185, 199)	1
  (185, 416)	1
  (185, 668)	1
  (185, 224)	1
  (185, 358)	1
  (185, 370)	1
  (185, 503)	1
  (185, 487)	1
  (185, 707)	1


In [95]:
from transformers import pipeline

In [96]:
rugen = pipeline('text-generation', model='sberbank-ai/rugpt3large_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [97]:
startText = codex

In [98]:
result = rugen(startText, max_length=500, temperature=0.5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [99]:
from transformers import AutoTokenizer

In [100]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [101]:
tokenizer(startText)

{'input_ids': [19, 40440, 815, 19, 75, 72, 86, 2217, 19, 48649, 43566, 2217, 19, 18442, 29, 2932, 6091, 292, 3174, 2467, 19, 71, 11082, 18, 88, 21388], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#-*- coding: utf-8 -*-
TOKEN = "6656889659:AAHGDYuByDyMVR-2z0GpNlmUlEE7Z6riHpw"
#https://t.me/s_consulting_bot
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np
import time
filename = "/content/gdrive/MyDrive/30092023хакатон/data.csv"
status = 0
import telebot
from telebot import types
bot = telebot.TeleBot(TOKEN)
@bot.message_handler(commands=["start"])
def start(m, res=False):
    answer = 'Здравствуйте, чем могу вам помочь ? '
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item_Accept_user_agreement = types.KeyboardButton("Переключить на оператора")
    markup.add(item_Accept_user_agreement)
    bot.send_message(m.chat.id, answer, reply_markup=markup)
# Получение сообщений от юзера
@bot.message_handler(content_types=["text"])
def handle_text(message):
    start_time = time.time()
    if message.text == "Переключить на оператора" :
        bot.send_message(810299040, text=str(message.from_user.id) + " : " + str(message.text))
        bot.send_message(810299040, text=str(message.from_user.first_name) + " : " + str(message.text))
        bot.send_message(message.chat.id, "Оператора ================= нету")

    else:
        bot.send_message(810299040, text=str(message.from_user.id) + " : " + str(message.text))
        bot.send_message(810299040, text=str(message.from_user.first_name) + " : " + str(message.text))
        def read_file(filename):
            global status
            read_pd = pd.read_csv(filename)
            print(read_pd)
            d_input = message.text

            def next_step_give_my_money(message):
                d_output = message.text

                def main(d_input, d_output):
                    data = open(filename, "a", encoding="utf-8")
                    data.write(str(time.strftime("%Y-%m-%d %H:%M:%S")) + ",")
                    data.write(str(d_input) + ",")
                    data.write(str(d_output) + "\n")
                    data.close()

                main(d_input, d_output)
            for i in range(read_pd.shape[0]):
                a = fuzz.token_sort_ratio(d_input,read_pd["question"][i])

                if a >= 90 :
                    print([i,a,read_pd["answer"][i]])
                    status = 1
                    bot.send_message(message.chat.id, read_pd["answer"][i])
            if status == 0:
                bot.send_message(message.chat.id, "Такого ответа нету, введите")
                bot.register_next_step_handler(message, next_step_give_my_money)

            status = 0
            end_time = time.time() - start_time
            bot.send_message(message.chat.id,"Качество представленного ответа: " + str("???") + "\n" + "Скорость получения ответа: " + str(end_time) + "\n" + "Устойчивость и масштабируемость системы :"+ str("???") + "\n" + "Способность к адаптации :"+ str("???")+ "\n" + "Автономность системы :"+ str("Полная"))

        read_file(filename)

# Запускаем бота
bot.polling(none_stop=True, interval=0)

               Datetime                                question  \
0   2023-09-29 02:33:51                             who are you   
1   2023-09-29 02:35:36                                  кто ты   
2   2023-09-29 02:35:43                                   кто я   
3   2023-09-29 02:43:09                                     кто   
4   2023-09-29 02:43:27                                     что   
5   2023-09-29 02:43:53                                  тко ты   
6   2023-09-29 02:48:38                          что происходит   
7   2023-09-29 14:40:00                                     а ?   
8   2023-09-29 14:45:13  сколько мне полагается отпускных дней?   
9   2023-09-29 15:02:51                 Что такое Смартократия?   
10  2023-09-29 17:02:55                                       k   
11  2023-09-29 17:22:30                                  отпуск   
12  2023-09-29 19:02:04                          какой вопрос ?   
13  2023-09-29 19:03:30                            что делать 